<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-02 10:19:19
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -63.38 K (-0.46%)
Current PnL: -14.16 L (-9.87%)
CY Booked + Current PnL: -7.42 L (-5.17%)
-------------------
Total profit:  4.34 L
Total loss:  -18.50 L
-------------------
Total Booked + Current PnL: 19.56 L (16.78%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.97%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.59 L (53.48%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,28.12,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.76,37.78,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,92.71,132894.0,-9585.0,10206.0,0.39,-6.73,7.68,0.44,243.0,-0.94,0.98,56.85,OX40N,NTT,BANKS
88,UNIONBANK,123.87,163.00,0.15,M-LC,42.71,175109.0,34269.0,10226.0,-0.41,24.33,5.84,31.59,89.0,3.35,1.29,52.11,XY24,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.08,H-LC,87.50,111264.0,6952.0,10581.0,0.16,6.66,9.51,16.81,27.0,0.66,0.82,23.78,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-17.40,H-LC,83.33,112508.0,8942.0,11768.0,0.06,8.63,10.46,20.00,30.0,0.76,0.83,23.92,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,100.00,140706.0,-1136.0,57704.0,-2.62,-0.80,41.01,39.88,259.0,-0.02,1.04,42.59,XY24,NTT,POWER
60,NATIONALUM,189.63,247.44,-49.40,H-MC,73.96,101458.0,954.0,29687.0,0.61,0.95,29.26,30.49,97.0,0.03,0.75,33.66,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,QUESS,296.83,986.00,-30.58,M-SC,41.67,69007.0,4001.0,146930.0,4.98,6.16,212.92,232.18,191.0,0.03,0.51,5.67,XY24,NTT,MISC
48,INFY,1461.46,2275.00,-8.47,X-LC,70.83,293059.0,31458.0,114176.0,1.89,12.03,38.96,55.67,2.0,0.28,2.16,18.86,X40,BTT,IT
83,TCS,3794.03,4998.00,-17.10,X-LC,47.92,270239.0,-25695.0,119608.0,1.02,-8.68,44.26,31.73,1.0,-0.21,1.99,8.17,X40,BTT,IT
17,BRITANNIA,4983.33,6446.05,16.31,X-LC,57.29,266570.0,37337.0,29936.0,0.90,16.29,11.23,29.35,10.0,1.25,1.96,26.66,XY25,ATH,FMCG
95,WIPRO,243.46,420.00,-3.31,M-LC,80.21,165416.0,14471.0,94982.0,0.85,9.59,57.42,72.51,69.0,0.15,1.22,13.94,XR,NTT,IT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,INDUSINDBK,1354.50,1800.0,79.16,L-MC,60.42,48441.0,-28766.0,54157.0,-3.33,-37.26,111.80,32.89,253.0,-0.53,0.36,33.40,XR,NTT,BANKS
92,VIPIND,488.80,489.0,-212.00,H-SC,96.88,80769.0,-14547.0,14587.0,-2.68,-15.26,18.06,0.04,154.0,-1.00,0.60,59.31,OX40N,NTT,MISC
52,JPPOWER,18.73,26.2,-30.08,L-SC,100.00,140706.0,-1136.0,57704.0,-2.62,-0.80,41.01,39.88,259.0,-0.02,1.04,42.59,XY24,NTT,POWER
30,DMART,3764.92,5201.0,-15.08,X-LC,82.29,213268.0,28787.0,41587.0,-2.25,15.60,19.50,38.14,15.0,0.69,1.57,27.86,X40N,NTT,FMCG
70,SATIN,199.77,274.0,-18.53,H-SC,78.12,161983.0,-33991.0,106812.0,-2.23,-17.34,65.94,37.16,130.0,-0.32,1.19,20.97,XY24,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,92.71,132894.0,-9585.0,10206.0,0.39,-6.73,7.68,0.44,243.0,-0.94,0.98,56.85,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-212.00,H-SC,96.88,80769.0,-14547.0,14587.0,-2.68,-15.26,18.06,0.04,154.0,-1.00,0.60,59.31,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-32.22,M-SC,6.25,144585.0,-26556.0,26502.0,-1.32,-15.52,18.33,-0.03,189.0,-1.00,1.07,3.92,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,123.77,H-SC,88.54,146556.0,-28003.0,28036.0,0.12,-16.04,19.13,0.02,127.0,-1.00,1.08,27.09,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.15,X-MC,16.15,85988.0,-16664.0,16656.0,-1.07,-16.23,19.37,-0.00,58.0,-1.00,0.63,30.71,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,41906.0,651.0,10154.0,-1.34,1.58,24.23,26.19,1.0,0.06,0.31,0.00,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.10,X-LC,47.92,270239.0,-25695.0,119608.0,1.02,-8.68,44.26,31.73,1.0,-0.21,1.99,8.17,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.47,X-LC,70.83,293059.0,31458.0,114176.0,1.89,12.03,38.96,55.67,2.0,0.28,2.16,18.86,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-18.00,X-LC,21.88,256592.0,-13755.0,48393.0,-0.20,-5.09,18.86,12.81,6.0,-0.28,1.89,7.31,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.84,X-LC,30.21,279560.0,14134.0,42773.0,-0.86,5.32,15.30,21.44,9.0,0.33,2.06,11.85,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,42.71,175109.0,34269.0,10226.0,-0.41,24.33,5.84,31.59,89.0,3.35,1.29,52.11,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,28.12,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.76,37.78,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,75.00,72725.0,38276.0,1673.0,-2.10,111.11,2.30,115.97,NaN,22.88,0.54,11.21,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.10,M-MC,52.08,229698.0,4736.0,162925.0,0.04,2.11,70.93,74.53,185.0,0.03,1.69,33.36,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,27.50,H-SC,69.79,130322.0,5025.0,54188.0,-2.21,4.01,41.58,47.25,139.0,0.09,0.96,63.04,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.87,H-SC,55.21,118930.0,16926.0,28103.0,0.10,16.59,23.63,44.14,124.0,0.60,0.88,35.53,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,28.12,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.76,37.78,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,42.71,175109.0,34269.0,10226.0,-0.41,24.33,5.84,31.59,89.0,3.35,1.29,52.11,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,75.00,72725.0,38276.0,1673.0,-2.10,111.11,2.30,115.97,NaN,22.88,0.54,11.21,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,91.67,202742.0,31121.0,130951.0,-0.01,18.13,64.59,94.43,198.0,0.24,1.49,49.64,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.31,M-LC,80.21,165416.0,14471.0,94982.0,0.85,9.59,57.42,72.51,69.0,0.15,1.22,13.94,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.33,H-SC,93.75,136899.0,13563.0,121265.0,-0.57,11.00,88.58,109.32,122.0,0.11,1.01,35.25,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,27.50,H-SC,69.79,130322.0,5025.0,54188.0,-2.21,4.01,41.58,47.25,139.0,0.09,0.96,63.04,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,97.92,68736.0,-44813.0,84861.0,-2.13,-39.47,123.46,35.27,261.0,-0.53,0.51,68.23,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.25,H-SC,66.67,92685.0,-5295.0,28028.0,-0.21,-5.40,30.24,23.21,137.0,-0.19,0.68,34.43,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,4.91,X-SC,98.96,97696.0,-3267.0,97794.0,-1.34,-3.24,100.10,93.63,64.0,-0.03,0.72,45.52,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,28.12,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.76,37.78,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-353.90,H-SC,5.21,108017.0,-25269.0,109864.0,0.31,-18.96,101.71,63.47,148.0,-0.23,0.80,81.06,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,100.00,140706.0,-1136.0,57704.0,-2.62,-0.80,41.01,39.88,259.0,-0.02,1.04,42.59,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,4.91,X-SC,98.96,97696.0,-3267.0,97794.0,-1.34,-3.24,100.10,93.63,64.0,-0.03,0.72,45.52,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,97.92,68736.0,-44813.0,84861.0,-2.13,-39.47,123.46,35.27,261.0,-0.53,0.51,68.23,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-212.00,H-SC,96.88,80769.0,-14547.0,14587.0,-2.68,-15.26,18.06,0.04,154.0,-1.00,0.60,59.31,OX40N,NTT,MISC
51,JIOFIN,310.58,387.00,6.36,H-LC,95.83,227920.0,10514.0,42986.0,-1.21,4.84,18.86,24.61,95.0,0.24,1.68,62.05,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.89
1,25,41.92
2,50,70.43


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.45
LC    34.15
MC    21.41
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.50
X40      12.31
XY25     12.01
X40N     11.79
XR       10.16
AR        8.24
OX40N     8.16
MH        1.71
X5K       1.48
X200      1.41
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.64
M-SC    16.02
H-LC    14.16
X-LC    12.24
H-MC     9.47
X-MC     7.50
M-LC     6.69
X-SC     4.34
M-MC     4.08
L-SC     2.45
L-LC     1.06
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.07,-2.22,36.39
IT,12.16,-10.20,64.34
BANKS,8.01,-4.78,47.54
FINANCE,7.88,-12.70,53.50
MISC,6.67,-15.24,68.71
PAINTS,5.65,-13.90,36.34
ELECTRICAL,5.47,0.67,45.72
HEALTHCARE,4.52,-5.11,35.81
AUTO,4.31,-15.41,66.16


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2729002.0,27
XR,1080023.0,13
AR,1026328.0,10
X40,650085.0,9
X40N,563037.0,11
XY25,488270.0,8
OX40N,322819.0,11
SR,179647.0,2
MH,83875.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1983873.0,20
M-SC,1770966.0,21
H-LC,618760.0,12
H-MC,548242.0,8
X-LC,512605.0,7
X-MC,434107.0,6
M-MC,415069.0,3
X-SC,329578.0,5
M-LC,266163.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      866742.0      6
M-SC       XY24      815559.0      7
H-SC       AR        523299.0      3
M-SC       XR        335814.0      4
M-MC       XY24      307763.0      2
H-MC       XY24      303267.0      3
X-LC       X40       233784.0      2
H-SC       X40N      228925.0      4
           XR        215964.0      3
H-LC       X40N      200269.0      4
M-SC       AR        182801.0      2
           OX40N     169567.0      5
H-LC       X40       159988.0      3
M-LC       XY24      158848.0      3
X-MC       X40       154450.0      2
L-SC       XR        147464.0      2
X-SC       XR        124336.0      1
X-LC       XY25      121102.0      3
           AR        116132.0      1
M-SC       XY25      113066.0      1
M-MC       XR        107306.0      1
X-MC       XY25      106406.0      1
           XY24       97918.0      1
X-SC       SR         97794.0      1
M-LC       XR         94982.0      1
H-SC       OX40N      94755.0      3
H-MC       XY25       89132.0      1
M-SC       SR         81853.0      1
H-LC       AR         79645.0      1
M-SC       X40        72306.0      1
H-LC       X200       68927.0      1
           X5K        66945.0      2
X-SC       XY24       65607.0      1
H-MC       AR         63020.0      1
L-SC       AR         59758.0      1
X-MC       X40N       58677.0      1
L-SC       XY24       57704.0      1
H-SC       MH         54188.0      1
L-MC       XR         54157.0      1
L-LC       XY25       46231.0      1
H-LC       XY24       42986.0      1
X-SC       OX40N      41841.0      2
X-LC       X40N       41587.0      1
H-MC       X40N       33579.0      1
           MH         29687.0      1
           X40        29557.0      1
X-MC       OX40N      16656.0      1
M-LC       XY25       12333.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
